# A1367 OC Photometry

This notebook gets the photometry for all sources in the image using automated detection
like SExtractor.

## Imports

In [ ]:
# Python Imports
import re
from os import path
from glob import iglob
from itertools import permutations as perms
import pickle

# Numerical Imports
import numpy as np

# Astropy Imports
from astropy import units as u
from astropy.io import fits
from astropy.wcs import WCS
from astropy.stats import SigmaClip
from astropy.convolution import Kernel2D, Gaussian2DKernel, convolve, convolve_fft
from photutils import background
from photutils.utils import calc_total_error
from photutils.segmentation import detect_sources, deblend_sources, SourceCatalog


## Notebook Setup

### Helper Classes

In [ ]:
# Image Data
# A container for relevant image data
class ImageData:

    # Constructor
    def __init__(
            self, fileName: str,
            sigmaClip: SigmaClip = SigmaClip(sigma=3.0, maxiters=10),
            kernel: Kernel2D = Gaussian2DKernel(2.47718858),  # FWHM = 3.5 FLC pix
            bg_box_size: int = 64,
            detection_threshold: float = 3.0,
            no_fft: bool = False,
            drop_nan: bool = True,
            imgUnit: u.Unit = u.electron / u.s
        ):

        # Store the File Name
        self.fileName = fileName

        # Store PhotUtils Parameters
        self._sigmaClip = sigmaClip
        self._kernel = kernel
        self._bg_box_size = bg_box_size
        self._detection_threshold = detection_threshold
        self.no_fft = no_fft
        self.drop_nan = drop_nan

        # Open the FITS File & Get Data
        with fits.open(fileName) as hduList:

            # Get the Primary Header
            self.primaryHeader = hduList[0].header

            # Get the Image Data
            self.imageHeader = hduList[1].header
            self.imageData = hduList[1].data << imgUnit
            self.imageWCS = WCS(self.imageHeader)

            # Get the Weight Data
            # We previously created weights as IVM in AstroDrizzle
            self.weightData = hduList[2].data << (imgUnit**-2)
            self.errorData = 1 / np.sqrt(self.weightData)

        # Get the Background
        self._background = self._get_background()

        # Get the Convolved Image
        self._convolvedData = self._convolve()

        # Get the Sources
        self._segmentImage = self._get_sources()

        # Get the Total Error
        self._totalError = self._calculate_total_error()

        # Get the Catalog
        self.catalog = self._get_catalog()


    # --- Helper Methods --- #
    # Background Subtraction
    def _get_background(self):
        return background.Background2D(
            self.imageData, self._bg_box_size,
            filter_size=5, sigma_clip=self._sigmaClip,
            bkg_estimator=background.SExtractorBackground(sigma_clip=self._sigmaClip)
        )

    # Convolution
    def _convolve(self):
        if np.any(np.array(self._kernel.shape) > 15) and not self.no_fft:
            return convolve_fft(self.residualImage, self._kernel, allow_huge=True)
        else:
            return convolve(self.residualImage, self._kernel)

    # Get Sources
    def _get_sources(self):

        # Get Original Source Map
        segMap = detect_sources(
            self._convolvedData,
            self._detection_threshold * self._background.background_rms,
            npixels=5
        )

        # Use Parallel Deblending?
        nproc = None if segMap.nlabels >= 1000 else 1

        # Deblend and Return
        return deblend_sources(
            self._convolvedData, segMap, npixels=5,
            nproc=nproc, progress_bar=False
        )

    # Calculate Total Error
    def _calculate_total_error(self):

        # RMS Error (Non-Poisson)
        rms = np.sqrt(self.errorData**2 + self._background.background_rms**2)

        # Return Total Error
        return calc_total_error(
            self.imageData, rms, effective_gain=self.primaryHeader['EXPTIME']*u.s
        )

    # Get Catalog
    def _get_catalog(self):

        # Get Catalog
        cat = SourceCatalog(
            self.imageData, self._segmentImage,
            convolved_data=self._convolvedData,
            error=self._totalError,
            background=self._background.background,
            wcs=self.imageWCS
        )

        # Drop NaN if Needed
        if self.drop_nan:
            cat = cat[~np.isnan(cat.segment_flux)]
            self._segmentImage.keep_labels(cat.labels)

        return cat


    # --- Public Methods --- #
    # Save to DS9
    def to_ds9(self, fileName: str, color: str = 'cyan', ellipse: bool = True):

        # Open the File
        with open(fileName, 'w') as fid:

            # Write the Header
            fid.write('# Region file format: DS9 version 4.1\n')
            fid.write(f'global color={color}\n')
            fid.write(f'{self.imageWCS.wcs.radesys}\n')

            if ellipse:

                # Write the Kron Apertures
                for src in self.catalog:

                    # Get Kron
                    kron = src.kron_aperture.to_sky(self.imageWCS)
                    ra   = kron.positions.ra.value
                    dec  = kron.positions.dec.value
                    a    = kron.a.to(u.arcsec).value
                    b    = kron.b.to(u.arcsec).value
                    theta= kron.theta.to(u.deg).value + 90
                    # Have to add 90 degrees to the theta to match DS9
                    # PhotUtils uses the angle from the x-axis, DS9 uses the angle from North

                    # Write the Ellipse
                    fid.write(f'ellipse({ra},{dec},{a}",{b}",{theta}) # text={{{src.label}}}\n')

            else:

                # Write the Sources
                for src in self.catalog:
                    ra  = src.sky_centroid.ra.value
                    dec = src.sky_centroid.dec.value
                    fid.write(f'point({ra},{dec}) # text={{{src.label}}}\n')

    # Use Other for Detection
    def use_other(self, other: 'ImageData'):

        return SourceCatalog(
            self.imageData, other.segmentImage,
            convolved_data=self._convolvedData,
            error=self._totalError,
            background=self._background.background,
            wcs=self.imageWCS, detection_cat=other.catalog
        )

    # -- Static Methods --- #
    @staticmethod
    def calculate_magnitude(flux: u.Quantity, zp: u.Quantity) -> u.Quantity:
        """Calculate the magnitude from the flux and zero point.

        Inputs
        ------
        flux : `~astropy.units.Quantity`
            The flux of the source in any unit that is not logarithmic.
        zp : `~astropy.units.Quantity`
            The zero point for the image in logarithmic units.
        """
        return u.Magnitude(flux) + zp

    # --- Property Methods --- #
    @property
    def background(self):
        return self._background

    @property
    def residualImage(self):
        return self.imageData - self._background.background

    @property
    def segmentImage(self):
        return self._segmentImage

    @property
    def zpST(self):
        zp = -2.5 * np.log10(self.imageHeader['PHOTFLAM']) - 21.1
        return u.Magnitude(zp, unit=u.mag(u.ST/self.imageData.unit))

    @property
    def zpAB(self):
        zp  = self.zpST.value
        zp += 18.6921
        zp -= 5 * np.log10(self.imageHeader['PHOTPLAM'])
        return u.Magnitude(zp, unit=u.mag(u.AB/self.imageData.unit))

    @property
    def pixelScale(self):
        return self.imageWCS.proj_plane_pixel_scales()[0].to(u.arcsec) / u.pix

    # --- Setter/Getter Methods --- #
    @property
    def sigmaClip(self):
        return self._sigmaClip

    @sigmaClip.setter
    def sigmaClip(self, value: SigmaClip):
        self._sigmaClip = value
        self._background = self._get_background()
        self._convolvedData = self._convolve()
        self._segmentImage = self._get_sources()
        self._totalError = self._calculate_total_error()
        self.catalog = self._get_catalog()

    @property
    def kernel(self):
        return self._kernel

    @kernel.setter
    def kernel(self, value: Kernel2D):
        self._kernel = value
        self._convolvedData = self._convolve()
        self._segmentImage = self._get_sources()
        self._totalError = self._calculate_total_error()
        self.catalog = self._get_catalog()

    @property
    def bg_box_size(self):
        return self._bg_box_size

    @bg_box_size.setter
    def bg_box_size(self, value: int):
        self._bg_box_size = value
        self._background = self._get_background()
        self._convolvedData = self._convolve()
        self._segmentImage = self._get_sources()
        self._totalError = self._calculate_total_error()
        self.catalog = self._get_catalog()

    @property
    def detection_threshold(self):
        return self._detection_threshold

    @detection_threshold.setter
    def detection_threshold(self, value: float):
        self._detection_threshold = value
        self._segmentImage = self._get_sources()
        self._totalError = self._calculate_total_error()
        self.catalog = self._get_catalog()

### Constants

In [ ]:
# Image Directory
IMAGE_DIR = '../Images/ProcessedImages/HST/DrizzledImages'

## Load Data

In [ ]:
# Load the Image Data for Each Image
imgDataDict = {}
for fileName in iglob(path.join(IMAGE_DIR, '*drc.fits')):
    match = re.search(r'F\d{3}W', fileName)
    filter = match.group()
    imgDataDict[filter] = ImageData(fileName, no_fft=True)

In [ ]:
imgData = imgDataDict['F814W']
cat = imgData.catalog

In [ ]:
fA, _ = cat.circular_photometry((0.5*u.arcsec/imgData.pixelScale).value)
fI, _ = cat.circular_photometry((5*u.arcsec/imgData.pixelScale).value)

In [ ]:
np.nanmedian(fA/fI), np.nanstd(fA/fI)

## Save Sources

### Save DS9 Regions

In [ ]:
%%bash
# Make the DS9 Catalog Directory
mkdir -p ../Images/ProcessedImages/HST/DS9/Catalogs

In [ ]:
# Write Catalogs to File
for filter, imgData in imgDataDict.items():
    imgData.to_ds9(path.join(
        IMAGE_DIR, '..', 'DS9', 'Catalogs',
        f'{filter}_Catalog.reg'
    ))

### Save Catalogs as Text File

In [ ]:
%%bash
# Make a Data/Catalog Directory
mkdir -p ../Data/Catalogs

In [ ]:
# Save the Catalogs
for filter, imgData in imgDataDict.items():

    imgData.catalog.to_table().write(f'../Data/Catalogs/{filter}_Catalog.ecsv', overwrite=True)

In [ ]:
# Get Cross-Detection Catalogs
for anFilt, detFilt in perms(imgDataDict.keys(), 2):

    # Get the Cross-Detection Catalog
    crossCat = imgDataDict[anFilt].use_other(imgDataDict[detFilt])

    # Save the Catalog
    crossCat.to_table().write(
        f'../Data/Catalogs/Detection-{detFilt}_Analysis-{anFilt}_Catalog.ecsv',
        overwrite=True
    )

## Calculate Magnitudes & Colors

* [Calculate Zero Points](https://spacetelescope.github.io/hst_notebooks/notebooks/WFC3/photometry_examples/phot_examples.html#example-1a-compute-the-inverse-sensitivity-and-zeropoint)
* [Calculate EE](https://spacetelescope.github.io/hst_notebooks/notebooks/WFC3/photometry_examples/phot_examples.html#example-1b-compute-an-encircled-energy-correction)
* Calculate MW Extinction

```python
ext = stsyn.ebmvx('xgalsb', ebv)
bp = stsyn.band(obsmode)
u.Magnitude(ext.integrate(bp.waveset) / np.trapz(x=bp.waveset,y=np.ones_like(bp.waveset).value)
```